# Libraries

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.optim import lr_scheduler
from torch.optim.lr_scheduler import ExponentialLR
from scipy.optimize import fsolve

# PINN network

In [ ]:
class PINN (nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        self.hidden_layers = nn.ModuleList([nn.Linear(2, 50)] + [nn.Linear(50, 50) for _ in range(7)])
        self.output_layer = nn.Linear(50, 1)
        self.activation = nn.Sigmoid()

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=1.0)
            if module.bias is not None:
                module.bias.data.zero_()

    def forward(self, x, t):
        torch.autograd.set_detect_anomaly(True)
        inputs = torch.cat([x, t], axis=1)
        outputs = self.activation(self.hidden_layers[0](inputs))
        for layer in self.hidden_layers[1:]:
            outputs = self.activation(layer(outputs))
        output = self.output_layer(outputs)
        return output

model1 = PINN()
optimizer = torch.optim.Adam(model1.parameters())

# Parameters

In [ ]:
Params = (0.0,)
v = Params[0] 
x0 = 0
x1 = 1
t0 = 0 
t1 = 0.15

# Points and Cost Functions

In [ ]:
def PDE_Data(x_0, x_1, t_0, t_1, n):
    # PDE points 
    xx = x_1 - x_0 
    # using Sobol sequence to achieve more even distribution 
    x = xx*torch.quasirandom.SobolEngine(dimension=1).draw(n, dtype=torch.float32) + x_0
    x.requires_grad_(False)

    t = torch.quasirandom.SobolEngine(dimension=1)
    t = t.draw(n, dtype=torch.float32)
    tt = t_1 - t_0
    t = tt * t
    t = t + t_0
    t.requires_grad_(False)


    PDE_points = torch.stack((x, t, torch.zeros(n,1,requires_grad=False)), dim=0)

    return PDE_points

def PDE_cost (Data,model,Param):
    x = Data[0,:]
    t = Data[1,:]

    x = x.clone().requires_grad_(True)
    t = t.clone().requires_grad_(True)
    u = model(x, t)
    du_dx = torch.autograd.grad(u.sum(), x, create_graph=True, retain_graph=True)[0]
    du_dt = torch.autograd.grad(u.sum(), t, create_graph=True, retain_graph=True)[0]
    d2u_dx2 = torch.autograd.grad(du_dx.sum(), x, create_graph=True, retain_graph=True)[0]

    PDE_loss = du_dt + u * du_dx - Param * d2u_dx2
    
    z = torch.nn.MSELoss()
    loss = z(PDE_loss,Data[2,:])
    
    return loss

In [ ]:
def lambda1 (x_BC,t_BC):
    return torch.zeros_like(x_BC,requires_grad=False)

def BC_Data (x_b,t_0,t_1,func,n):
    x_BC = torch.ones(n,1,requires_grad=False)
    x_BC = x_BC * x_b

    t_collocation_Sobol = torch.quasirandom.SobolEngine(dimension=1)
    t_collocation1 = t_collocation_Sobol.draw(n,dtype=torch.float32)
    tt = t_1 - t_0
    t_BC = tt * t_collocation1
    t_BC = t_BC + t_0
    t_BC.requires_grad_(False)

    v = func(x_BC,t_BC)
    BC_points = torch.stack((x_BC,t_BC,v),axis=0)
    return BC_points

In [ ]:
def lambda2 (x_IC,t_IC):
    return torch.sin(torch.pi * 2 * x_IC).requires_grad_(False)

def IC_Data (x_0,x_1,t_i,func,n):
    
    x_sobol = torch.quasirandom.SobolEngine(dimension=1)
    x_IC = x_sobol.draw(n,dtype=torch.float32)
    xx = x_1 - x_0
    x_IC = x_IC * xx
    x_IC = x_IC + x_0
    x_IC.requires_grad_(False)

    t_IC = torch.ones(n,1,requires_grad=False)
    t_IC = t_IC * t_i

    v = func(x_IC,t_IC)

    IC_points = torch.stack((x_IC,t_IC,v),axis=0)
    return IC_points

In [ ]:
def cost (Data,model):
    
    #define the cost method 
    cost_function = torch.nn.MSELoss()
    return cost_function(Data[2,:],model(Data[0,:],Data[1,:]))

# Analythical Solution

In [ ]:
def Answers_equation (u,x,t):
    return u - np.sin(2*np.pi*(x-u*t))

def validation (x,t,func,n):
    t_value = t
    x1 = x
    x1 = np.sort(x1)
    x1 = x1.reshape(n,1)
    solution = np.zeros(x1.shape)
    x_values = np.zeros(x1.shape)

    for i , x_values in enumerate(x1[1:]):
        solution[i] = fsolve(func,solution[i-1],args=(x_values,t_value))

    print(x_values.shape)
    
    for i,x_values in enumerate(x1):
        print (f"solution at x={x_values} and t={t_value} is u={solution[i]}")
   
    return solution

# Train

In [ ]:
iterations = 10000

BC1_Points = BC_Data(x0,t0,t1,lambda1,500)
BC2_Points = BC_Data(x1,t0,t1,lambda1,500)
IC_Points = IC_Data(x0,x1,t0,lambda2,500)
PDE_Points = PDE_Data(x0,x1,t0,t1,3000)

optimizer = torch.optim.Adam(model1.parameters(), lr=0.001)

for epoch in range (iterations):
    optimizer.zero_grad()
    torch.autograd.set_detect_anomaly(True)
    #BC loss
    BC1_loss = cost(BC1_Points,model1)
    BC2_loss = cost(BC2_Points,model1)

    #IC loss
    IC_loss = cost(IC_Points,model1)

    #PDE loss
    #PDE_Points = PDE_Data(0,1,0,0.5,model1,v,100)
    #PDE_loss = PDE_Points[2,:,0].reshape(100,1)
    PDE_loss = PDE_cost(PDE_Points,model1,v)

    #total loss
    loss = BC1_loss + BC2_loss + IC_loss + PDE_loss

    loss.backward(retain_graph=True)
    optimizer.step()

    with torch.autograd.no_grad():
        print(epoch,"Training Loss:",loss.data)

# Plotting Results

In [ ]:
data1 = PDE_Data(0,1,0,0.1,500)
x1 = data1[0,:,0].detach().numpy()
t1 = 0.1
analytical_solution = validation(x1,t1,Answers_equation,500)
x1 = np.sort(x1)

# Plot the analythical answer graph only
plt.plot(x1, analytical_solution, label='Function Curve')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('Graph of Y vs. X')
plt.legend()
plt.show() 

x3 = torch.linspace(0,1,100).reshape(100,1)
t = 0.1 * torch.ones(100,1,requires_grad=False).reshape(100,1)

final = model1(x3,t)

x_graph = x3.detach().numpy()
answer_graph = final.detach().numpy()

# Plot the model and analythical answer graph
plt.plot(x3, answer_graph, label='Function Curve')
plt.plot(x1, analytical_solution, label='Analytical Curve')
plt.scatter(x3, answer_graph, color='red', label='Data Points')  # Scatter plot for data points
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('Graph of Y vs. X')
plt.legend()
plt.show()